# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.234269534680                   -0.50    8.0         
  2   -7.250226957832       -1.80       -1.41    1.0   7.91ms
  3   -7.251152838889       -3.03       -2.25    1.0   7.71ms
  4   -7.251318223011       -3.78       -2.36    3.0   10.9ms
  5   -7.251334667834       -4.78       -2.74    1.0   7.86ms
  6   -7.251337296518       -5.58       -3.07    1.0   7.80ms
  7   -7.251338716657       -5.85       -3.64    1.0   8.10ms
  8   -7.251338787055       -7.15       -3.97    2.0   9.65ms
  9   -7.251338796907       -8.01       -4.45    1.0   8.29ms
 10   -7.251338798499       -8.80       -4.93    3.0   11.1ms
 11   -7.251338798692       -9.72       -5.44    2.0   10.0ms
 12   -7.251338798703      -10.95       -5.97    1.0   8.44ms
 13   -7.251338798704      -11.91       -6.40    2.0   10.4ms
 14   -7.251338798704      -12.89       -6.78    1.0   8.54ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.03273661113139435
[ Info: Arnoldi iteration step 2: normres = 0.6738593409539334
[ Info: Arnoldi iteration step 3: normres = 0.7567370577686792
[ Info: Arnoldi iteration step 4: normres = 0.48318389417575575
[ Info: Arnoldi iteration step 5: normres = 0.32142225175456485
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (5.38e-02, 7.83e-02, 1.80e-01, 2.41e-01, 6.46e-02)
[ Info: Arnoldi iteration step 6: normres = 0.39924077217098997
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.82e-02, 1.30e-01, 3.66e-01, 6.73e-02, 3.20e-02)
[ Info: Arnoldi iteration step 7: normres = 0.09478745462826177
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (9.21e-04, 2.12e-02, 2.57e-02, 3.05e-02, 7.62e-02)
[ Info: Arnoldi iteration step 8: normres = 0.13174209783528354
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (5.21